In [4]:
# Import libraries and  modules
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam


In [5]:
# Set the folder path 
folder = "/kaggle/input/caltech101/101_ObjectCategories"


In [17]:
# Define function to load images from folder

def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    labels = []
    selected_classes = {
        "non-faulty": "airplanes",  # Using the "airplanes" class as an example for non-faulty products
        "faulty": "Faces_easy"      # Using the "Faces_easy" class as an example for faulty products
    }
    
    for label, class_name in selected_classes.items():
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img = cv2.imread(os.path.join(class_folder, filename))
            if img is not None:
                img_resized = cv2.resize(img, target_size)
                img_normalized = img_resized.astype("float32") / 255.0
                images.append(img_normalized)
                labels.append(label)
    return images, labels


In [18]:
# Load and preprocess the dataset

images, labels = load_images_from_folder(folder)
images = np.array(images) / 255.0

if len(labels) == 0:
    print("No data found. Please check the dataset path.")
else:
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    num_classes = np.max(integer_encoded) + 1 if integer_encoded.size > 0 else 0
    labels = to_categorical(integer_encoded, num_classes=num_classes)

In [19]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [20]:
# Define the CNN model
def create_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]
model = create_cnn(input_shape, num_classes)


In [22]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [23]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 4s 48ms/step - loss: 0.6535 - accuracy: 0.6405 - val_loss: 0.6380 - val_accuracy: 0.6313
Epoch 2/20
25/25 [==============================] - 1s 22ms/step - loss: 0.5021 - accuracy: 0.7734 - val_loss: 0.2825 - val_accuracy: 0.9495
Epoch 3/20
25/25 [==============================] - 1s 21ms/step - loss: 0.2503 - accuracy: 0.9456 - val_loss: 0.1264 - val_accuracy: 0.9646
Epoch 4/20
25/25 [==============================] - 1s 22ms/step - loss: 0.1164 - accuracy: 0.9684 - val_loss: 0.0667 - val_accuracy: 0.9798
Epoch 5/20
25/25 [==============================] - 1s 22ms/step - loss: 0.0626 - accuracy: 0.9810 - val_loss: 0.0410 - val_accuracy: 0.9848
Epoch 6/20
25/25 [==============================] - 1s 22ms/step - loss: 0.0521 - accuracy: 0.9810 - val_loss: 0.0532 - val_accuracy: 0.9798
Epoch 7/20
25/25 [==============================] - 1s 21ms/step - loss: 0.0681 - accuracy: 0.9747 - val_loss: 0.0332 - val_accuracy: 0.9949
Epoch 8/20
25

In [24]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")


8/8 [==============================] - 0s 34ms/step - loss: 0.0620 - accuracy: 0.9757
Test loss: 0.062037065625190735, Test accuracy: 0.9757084846496582


In [25]:
# Load and preprocess image
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    if img is not None:
        img_resized = cv2.resize(img, target_size)
        img_normalized = img_resized.astype("float32") / 255.0
        return img_normalized
    return None


In [30]:
# Path to test image
test_image_path = "/kaggle/input/caltech101/101_ObjectCategories/Faces_easy/image_0055.jpg"
test_image = load_and_preprocess_image(test_image_path)

In [31]:
# Use the trained model to make predictions on the preprocessed image

# Add a batch dimension to the image (the model expects a batch of images as input)
test_image_batch = np.expand_dims(test_image, axis=0)

# Make a prediction
predictions = model.predict(test_image_batch)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])

# Decode the predicted class back to its label
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

print(f"Predicted label: {predicted_label}")


1/1 [==============================] - 0s 23ms/step
Predicted label: faulty


In [32]:
# Path to test image
test_image_path = "/kaggle/input/caltech101/101_ObjectCategories/airplanes/image_0020.jpg"
test_image = load_and_preprocess_image(test_image_path)

In [33]:
# Use the trained model to make predictions on the preprocessed image

# Add a batch dimension to the image (the model expects a batch of images as input)
test_image_batch = np.expand_dims(test_image, axis=0)

# Make a prediction
predictions = model.predict(test_image_batch)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])

# Decode the predicted class back to its label
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

print(f"Predicted label: {predicted_label}")


1/1 [==============================] - 0s 22ms/step
Predicted label: non-faulty
